# --- `Day 25`: Sea Cucumber ---

In [2]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache
from ipycanvas import RoughCanvas, hold_canvas, Canvas
from time import sleep

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [3]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [4]:
final_input = parse_input(aocd.get_data(day=25, year=2021))
print(final_input[:5])

['>>vv..>v..>vv.....>vv...>.>...>>......v>v>.v.v..>v..v>.>v.....v..>vvvv.>.>vv>v..v>.v>..v>>>>.>vv.>>vv>.>>v..>.v.v>v>v.>vv.>.......vv.>.>>v.', 'v>.>v>.v....>>v.vvv>.>>..>....>.v>.v>vv.v..v.v..v>.>.v>.>..>>v.vvvvv..>>..v.v..>>v>vv>.v..vvv.>v..v>>>v.v..>>...>vv..>>v.v>>.>v>.>.v....>>.', 'v.>vv....v>..v>>..v.>>.....vv..>.vvv>.....v.v>vvv>..v>>..v...>...>...vv.v..>.vvv>>.v>vv..v.>>>....vv.>.>..v..vv.>>v>.>>.>>.v>>>>.>v.v.>v..v', '.v.>>..v...v>v.>>>>v....>.....>...v.v..>.v...>.v.....>>.v>v...>.>.>..>v>>>>.v.>..>......>.v>>.>v..>.>vv.>.v>...v.>v.>>.>...>>>..>>>.vv>..v.', '.....v.>.v>........>>.>.>vvv.vv>>.>vvv.v.>vvv.....v..>v..>vvv.>.v>.>..v..vv.>...v>>>>vv>.v..v.>..>..vvvv.>.>vvv....v>..>...v...v.v>v.>..v.v']


In [5]:
test_input = parse_input('''\
v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>
''')

print(test_input)

['v...>>.vv>', '.vv>>.vv..', '>>.>v>...v', '>>v>>.>.v.', 'v>v.vv.v..', '>.>>..v...', '.vv..>.>v.', 'v.v..>>v.v', '....v..v.>']


### Helpers

In [6]:
def printBoard(board, marked = set()):
    spacing = 1
    for r in range(min(20, len(board))):
        print(''.join(str.center(str(board[r][c]), spacing)
                if (r,c) not in marked else str.center(f"({board[r][c]})", spacing) 
                for c in range(min(20, len(board[0])))))
    print("")

def createCanvas(w, h, size):
    if size > 10:
        canvas = RoughCanvas(width = w, height = h)
    else:
        canvas = Canvas(width = w, height = h)
    canvas.font = '18px serif'
    canvas.fill_style = "#FFF0C9"
    canvas.stroke_style = "white"
    canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

    return canvas

## Solution 1

In [7]:
def drawAsText(board, canvas, pixelSize, count):
    color = "#5770B3"

    canvas.clear()
    canvas.fill_style = "#FFF0C9"
    canvas.stroke_style = "white"
    canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

    canvas.fill_style = color
    canvas.stroke_style = color

    w,h = len(board[0]), len(board)
    offset = 50

    for r in range(h):
        for c in range(w):
            if board[r][c] == ">":
                canvas.fill_text(">", c * pixelSize + 3 + offset, r * pixelSize + pixelSize - 2, pixelSize)
            elif board[r][c] == "v":
                canvas.fill_text("v", c * pixelSize + 3 + offset, r * pixelSize + pixelSize - 2, pixelSize)

    canvas.fill_style = "black"
    canvas.fill_text(str(count), 10, 20)

def drawAsRect(board, canvas, pixelSize, count):
    w,h = len(board[0]), len(board)

    offset = 50

    canvas.clear()
    canvas.fill_style = "#FFF0C9"
    canvas.stroke_style = "white"
    canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

    canvas.fill_style = "#5770B3"
    canvas.stroke_style = "#5770B3"
    xs = [i * pixelSize + offset for r in board for i,c in enumerate(r) if c == '>']
    ys = [i * pixelSize for i,r in enumerate(board) for c in r if c == '>']
    canvas.fill_rects(xs, ys, pixelSize)
    canvas.stroke_rects(xs, ys, pixelSize)

    canvas.fill_style = "#577073"
    canvas.stroke_style = "#577073"
    xs = [i * pixelSize + offset for r in board for i,c in enumerate(r) if c == 'v']
    ys = [i * pixelSize for i,r in enumerate(board) for c in r if c == 'v']
    canvas.fill_rects(xs, ys, pixelSize)
    canvas.stroke_rects(xs, ys, pixelSize)

    canvas.fill_style = "black"
    canvas.fill_text(str(count), 10, 20)

def draw(board, canvas, pixelSize, count):
    if pixelSize < 10:
        drawAsRect(board, canvas, pixelSize, count)
    else:
        drawAsText(board, canvas, pixelSize, count)

def solve_1(input):
    board = [line for line in input]
    w,h = len(board[0]), len(board)

    pixelSize = 15
    if w > 50:
        pixelSize = 2
    canvas = createCanvas(w * pixelSize + 50, h * pixelSize, pixelSize)
    display(canvas)
    draw(board, canvas, pixelSize, 0)
    
    count = 0
    while True:
        with hold_canvas(canvas):
            count += 1
            step1 = [[board[r][c] for c in range(w)] for r in range(h)]

            moved = False
            for r in range(h):
                for c in range(w):
                    if board[r][c] == '>':
                        if board[r][(c + 1) % w] == '.':
                            moved = True
                            step1[r][(c + 1) % w] = '>'
                            step1[r][c] = '.'

            board = step1
            step2 = [[board[r][c] for c in range(w)] for r in range(h)]
            for r in range(h):
                for c in range(w):
                    if board[r][c] == 'v':
                        if board[(r + 1) % h][c] == '.':
                            moved = True
                            step2[(r + 1) % h][c] = 'v'
                            step2[r][c] = '.'
            board = step2
            draw(board, canvas, pixelSize, count)
            if pixelSize >= 10:
                sleep(0.1)
            #printBoard(board)
            if not moved:
                #printBoard(step2)
                return count

solve_1(test_input)

RoughCanvas(height=135, width=200)

58

In [8]:
f"Solution 1: {solve_1(final_input)}"

Canvas(height=274, width=328)

'Solution 1: 456'